In [26]:
import lightgbm
import numpy
import pandas
import random
import os
import numpy as np
def statfeature(table, key, statdict, pre=""):
   if not isinstance(key, list):
      key = [key]
   table_temp=table.groupby(key).aggregate(statdict)
   table_temp.columns = ["%s%s and %s%s" % (pre, "".join(key), col, c if isinstance(c, str) else c.__name__) for col, fun in statdict.items() for c in (fun if isinstance(fun, list) else [fun])]
   return table_temp
pandas.DataFrame.statfeature = statfeature

train_table = pandas.read_csv("final_dataset_train.tsv", sep="\t")
train_table["id"] = range(-1, -len(train_table) -1, -1)
test_table = pandas.read_csv("final_dataset_testA.tsv", sep="\t")
test_table["delta_g"] = -1

In [27]:
def nature_vector(W):
    N=len(W)
    n=np.zeros(20)
    u=np.zeros(20)
    d=np.zeros(20)
    D=np.zeros(20)
    s=np.zeros((20,N))
    for i in range(N):
        if W[i]=='A':
            n[0]=n[0]+1
            s[0][int(n[0])]=i+1
        if W[i]=='C':
            n[1]=n[1]+1
            s[1][int(n[1])]=i+1
        if W[i]=='D':
            n[2]=n[2]+1
            s[2][int(n[2])]=i+1
        if W[i]=='E':
            n[3]=n[3]+1
            s[3][int(n[3])]=i+1
        if W[i]=='F':
            n[4]=n[4]+1
            s[4][int(n[4])]=i+1
        if W[i]=='G':
            n[5]=n[5]+1
            s[5][int(n[5])]=i+1
        if W[i]=='H':
            n[6]=n[6]+1
            s[6][int(n[6])]=i+1
        if W[i]=='I':
            n[7]=n[7]+1
            s[7][int(n[7])]=i+1
        if W[i]=='K':
            n[8]=n[8]+1
            s[8][int(n[8])]=i+1
        if W[i]=='L':
            n[9]=n[9]+1
            s[9][int(n[9])]=i+1
        if W[i]=='M':
            n[10]=n[10]+1
            s[10][int(n[10])]=i+1
        if W[i]=='N':
            n[11]=n[11]+1
            s[11][int(n[11])]=i+1
        if W[i]=='P':
            n[12]=n[12]+1
            s[12][int(n[12])]=i+1
        if W[i]=='Q':
            n[13]=n[13]+1
            s[13][int(n[13])]=i+1
        if W[i]=='R':
            n[14]=n[14]+1
            s[14][int(n[14])]=i+1
        if W[i]=='S':
            n[15]=n[15]+1
            s[15][int(n[15])]=i+1
        if W[i]=='T':
            n[16]=n[16]+1
            s[16][int(n[16])]=i+1
        if W[i]=='V':
            n[17]=n[17]+1
            s[17][int(n[17])]=i+1
        if W[i]=='W':
            n[18]=n[18]+1
            s[18][int(n[18])]=i+1
        if W[i]=='Y':
            n[19]=n[19]+1
            s[19][int(n[19])]=i+1
    for j in range(20):
        if n[j]>0:
            u[j]=sum(s[j,:])/n[j]
        if n[j]==0:
            u[j]=0
        t=0
        for l in range(int(n[j]+1)):
            t=t+(s[j,l]-u[j])*(s[j,l]-u[j])
        if n[j]>0:
            D[j]=t/(n[j]*N)
        if n[j]==0:
            D[j]=0
    for m in range(20):
        d[m]=D[m]
    h=np.append(n,u)
    vector=np.append(h,d)
    return vector


In [28]:
train_base_feature_table = train_table.loc[:, ["id","pdb", "antibody_seq_a", "antibody_seq_b", "antigen_seq"]]
for a in ["antibody_seq_a", "antibody_seq_b", "antigen_seq"]:
    train_base_feature_table["%s length" % a] = train_base_feature_table[a].str.len()
    ll=train_base_feature_table[a].str.len()
    feature=np.zeros((1706,60))
    feature_structure=np.zeros((1706,900))
    for b in range(1706):
        fasta=train_base_feature_table[a][b]
        feature[b,:]=nature_vector(fasta)
        try:
            pdb_name=train_base_feature_table["pdb"][b]
            feature_structure_name=os.path.join("./feature_file/",str(b)+'_'+pdb_name+'_'+a+'_feature.txt')
            feature_structure_np=np.loadtxt(feature_structure_name)
            feature_temp=feature_structure_np.reshape((1,900))
            feature_structure[b,:]=feature_temp
        except:
            print(pdb_name)
    for c in range(60):
        train_base_feature_table["%s_%s" % (a, c)] = feature[:,c]
    for t in range(900):
        train_base_feature_table["%s_%s structure" % (a, t)] = feature_structure[:,t]
   # for m in [chr(65 + x) for x in range(26)]:
     #   train_base_feature_table["%s_%s" % (a, m)] = train_base_feature_table[a].str.count(m)
     #   for n in [chr(65 + x) for x in range(26)]:
      #      train_base_feature_table["%s_%s" % (a, m + n)] = train_base_feature_table[a].str.count(m + n)
        #    for p in [chr(65 + x) for x in range(26)]:
         #       train_base_feature_table["%s_%s" % (a, m + n + p)] = train_base_feature_table[a].str.count(m + n + p)
train_base_feature_table = train_base_feature_table.drop(["pdb","antibody_seq_a", "antibody_seq_b", "antigen_seq"], axis=1)
test_base_feature_table = test_table.loc[:, ["id","pdb", "antibody_seq_a", "antibody_seq_b", "antigen_seq"]]
for a in ["antibody_seq_a", "antibody_seq_b", "antigen_seq"]:
    test_base_feature_table["%s length" % a] = test_base_feature_table[a].str.len()
    ll=test_base_feature_table[a].str.len()
    feature=np.zeros((178,60))
    feature_structure=np.zeros((178,900))
    for b in range(178):
        fasta=test_base_feature_table[a][b]
        feature[b,:]=nature_vector(fasta)
        try:
            pdb_name=train_base_feature_table["pdb"][b]
            feature_structure_name=os.path.join("./feature_file/",str(b)+'_'+pdb_name+'_'+a+'_feature.txt')
            feature_structure_np=np.loadtxt(feature_structure_name)
            feature_temp=feature_structure_np.reshape((1,900))
            feature_structure[b,:]=feature_temp
        except:
            print(pdb_name)
            
    for c in range(60):
        test_base_feature_table["%s_%s" % (a, c)] = feature[:,c]
    for t in range(900):
        test_base_feature_table["%s_%s structure" % (a, t)] = feature_structure[:,t]
  #  for m in [chr(65 + x) for x in range(26)]:
  #      test_base_feature_table["%s_%s" % (a, m)] = test_base_feature_table[a].str.count(m)
   #     for n in [chr(65 + x) for x in range(26)]:
   #         test_base_feature_table["%s_%s" % (a, m + n)] = test_base_feature_table[a].str.count(m + n)
     #       for p in [chr(65 + x) for x in range(26)]:
       #         test_base_feature_table["%s_%s" % (a, m + n + p)] = test_base_feature_table[a].str.count(m + n + p)
test_base_feature_table = test_base_feature_table.drop(["pdb","antibody_seq_a", "antibody_seq_b", "antigen_seq"], axis=1)

1EO8
1FE8
1NMA
1QFU
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83
1T83


6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK
6MLK


In [29]:
test_base_feature_table

,id,antibody_seq_a length,antibody_seq_a_0,antibody_seq_a_1,antibody_seq_a_2,antibody_seq_a_3,antibody_seq_a_4,antibody_seq_a_5,antibody_seq_a_6,antibody_seq_a_7,...,antigen_seq_890 structure,antigen_seq_891 structure,antigen_seq_892 structure,antigen_seq_893 structure,antigen_seq_894 structure,antigen_seq_895 structure,antigen_seq_896 structure,antigen_seq_897 structure,antigen_seq_898 structure,antigen_seq_899 structure
0,0,214,8.0,5.0,11.0,10.0,9.0,13.0,4.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,214,8.0,5.0,11.0,10.0,9.0,13.0,4.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,214,9.0,5.0,11.0,10.0,9.0,13.0,4.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,210,9.0,5.0,8.0,5.0,3.0,15.0,3.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,210,10.0,5.0,8.0,5.0,3.0,15.0,3.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,220,14.0,5.0,8.0,5.0,7.0,14.0,2.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,116,4.0,2.0,7.0,3.0,2.0,12.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,218,15.0,5.0,9.0,9.0,6.0,18.0,3.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,218,16.0,5.0,6.0,9.0,6.0,19.0,3.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,218,15.0,5.0,7.0,9.0,6.0,19.0,3.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
def extract_table(table, base_feature_table, feature_table):
   table_temp = table
   table_temp = table_temp.merge(base_feature_table, on="id", how="left")
   table_temp = table_temp.merge(feature_table.statfeature("antibody_seq_a", {"delta_g": ["mean", "median", "min", "max"]}).reset_index(), on="antibody_seq_a", how="left")
   table_temp = table_temp.merge(feature_table.statfeature("antibody_seq_b", {"delta_g": ["mean", "median", "min", "max"]}).reset_index(), on="antibody_seq_b", how="left")
   table_temp = table_temp.merge(feature_table.statfeature("antigen_seq", {"delta_g": ["mean", "median", "min", "max"]}).reset_index(), on="antigen_seq", how="left")
   table_temp = table_temp.drop(["pdb", "antibody_seq_a", "antibody_seq_b", "antigen_seq"], axis=1)

   table_temp["label"] = table_temp.delta_g.rank()
   table_temp = table_temp.loc[:, ["id", "delta_g", "label"] + [x for x in table_temp.columns if x not in ["id", "delta_g", "label"]]]

   return table_temp

In [31]:
foldN = 6
index = random.sample(range(len(train_table)), len(train_table))
train_data_table = None
for y in range(foldN):
   y_label_table = train_table.iloc[[x for x in range(len(index)) if x % foldN == y]].reset_index(drop=True)
   y_feature_table = train_table.iloc[[x for x in range(len(index)) if x % foldN != y]].reset_index(drop=True)

   y_data_table = extract_table(y_label_table, train_base_feature_table, y_feature_table)
   train_data_table = pandas.concat([train_data_table, y_data_table], ignore_index=True)
lightmodel = lightgbm.train(train_set=lightgbm.Dataset(train_data_table.iloc[:, 3:], label=train_data_table.label)
   , num_boost_round=2048, params={"objective": "regression", "learning_rate": 0.02, "max_depth": 6, "num_leaves": 32, "bagging_fraction": 0.7, "feature_fraction": 0.7, "num_threads": 64, "verbose": -1}
)
test_data_table = extract_table(test_table, test_base_feature_table, train_table)

predcit_table = test_data_table.loc[:, ["id"]]
predcit_table["delta_g"] = lightmodel.predict(test_data_table.iloc[:, 3:])

predcit_table.to_csv("1020_1_base_nature.csv", index=False)